In [1]:
import os
import sys
import json

In [2]:
from openapi_typed_2 import convert_to_openapi

In [3]:
sample_file = 'openapi_specs/original.json'
relpath = os.path.relpath(sample_file)

In [4]:
with open(relpath, 'r') as f:
    data = json.load(f)
    #specs = convert_to_openapi(data)

In [5]:
#paths = []

In [6]:
paths = []
for key in data['paths']:
    paths.append(key)

In [7]:
#path1 = paths[all_paths[2]]
#path2 = paths[all_paths[1]]

In [8]:
#path1.get.responses['200'].content['application/json'].schema.properties['accounts'].items.properties['accountId']._type                                         

In [9]:
acc2 = data['paths'][paths[1]]['get']['responses']['200']['content']['application/json']['schema']

In [10]:
acc1 = data['paths'][paths[2]]['get']['responses']['200']['content']['application/json']['schema']

In [11]:
pay1 =  data['paths'][paths[0]]['post']['responses']['200']['content']['application/json']['schema']

In [12]:
from schema_paths.schema_to_vector import generate_schema_vectors
from schema_paths.schema_to_fields import schema_remove_types

In [13]:
schemas = 'sample_jsonschemas/account_1.jsonl'
relschemas = os.path.relpath(schemas)

In [14]:
with open(relschemas, 'r') as f:
    data2 = json.load(f)

In [15]:
acc1['$schema'] = 'root'
acc2['$schema'] = 'root'
pay1['$schema'] = 'root'

In [16]:
a2 = generate_schema_vectors(acc2)


In [17]:
a1 = generate_schema_vectors(acc1)

In [18]:
a1 = [item.split('#') for item in a1]

In [19]:
d1 = dict()

In [20]:
for each in a1:
    length = str(len(each))
    each = '#'.join(each)
    if d1.get(length) is None:
        d1[length] = [each]
    else:
        d1[length] += [each]

In [21]:
level_keys = dict()
total_keys = len(d1)
for i in range(1, total_keys, 1):
    for each in d1[str(i)]:
        if each.endswith('@array') or each.endswith('@object'):
            if level_keys.get(str(i+1)) is None:
                level_keys[str(i+1)] = [each]
            else:
                level_keys[str(i+1)] += [each]

In [22]:
level_keys['4']

['accounts@array#_links@object#self@object',
 'accounts@array#_links@object#transactions@object']

In [23]:
total_keys = len(d1)
for i in range(total_keys, 1, -1):
    d1[str(i - 1)] += d1[str(i)]

In [24]:
l2 = list()

In [25]:
l2.append({'$schema' : d1['1']})

In [26]:
temp = {}
for each in level_keys['2']:
    for value in d1['2']:
        if value.startswith(each):
            value = value.replace(each, '')[1:]
            if temp.get(each) is None:
                temp[each] = [value]
            else:
                temp[each] += [value]
                
                

In [27]:
l2 = list()
for i in range(1, total_keys+1, 1):
    if i == 1:
        l2.append({ '$schema' : d1[str(i)]})
    else:
        temp = dict()
        for each in level_keys[str(i)]:
            temp[each] = []
            for value in d1[str(i)]:
                if value.startswith(each):
                    value = value.replace(each, '')[1:]
                    temp[each] += [value]
        l2.append(temp)

In [28]:
l2[1]

{'accounts@array': ['accountId@string',
  'identifier@string',
  'identifierScheme@string',
  'nickname@string',
  'name@string',
  'balance@number',
  'currency@string',
  'servicerScheme@string',
  'servicerIdentifier@string',
  '_links@object',
  '_links@object#self@object',
  '_links@object#transactions@object',
  '_links@object#self@object#href@string',
  '_links@object#transactions@object#href@string'],
 '_links@object': ['self@object', 'self@object#href@string']}

In [29]:
from schema_paths.parse_openapi_schema import parse_schema

In [30]:
a1 = parse_schema(acc1)

In [31]:
a2 = parse_schema(acc2)

In [35]:
if a2[0]['$schema'] == a1[1]['accounts@array']:
    print('Matched')
else:
    print('Unmatched')

Matched


In [43]:
match_list = []
for each1 in a2:
    for key1, value1 in each1.items():
        for each2 in a1:
            for key2, value2 in each2.items():
                if value1 == value2:
                    match_list.append((key1, key2))

In [44]:
for each in match_list:
    print(each)

('$schema', 'accounts@array')
('_links@object', 'accounts@array#_links@object')
('_links@object#self@object', '_links@object#self@object')
('_links@object#self@object', 'accounts@array#_links@object#self@object')
('_links@object#self@object', 'accounts@array#_links@object#transactions@object')
('_links@object#transactions@object', '_links@object#self@object')
('_links@object#transactions@object', 'accounts@array#_links@object#self@object')
('_links@object#transactions@object', 'accounts@array#_links@object#transactions@object')
